In [2]:
import pandas as pd
import nltk
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv ("Classeur1.csv",on_bad_lines='skip')
df
df.head()

,classe,paragraph
0,False,if you live in the european region whatsapp ir...
1,False,whatsapp legal info
2,False,if you live outside the european region whatsa...
3,False,for example our privacy policy talks about wha...
4,False,we are one of the facebook companies you can l...


In [5]:
# turn the classes and paragraphs into lists 
classes = df['classe'].tolist()
paragraphs = df['paragraph'].tolist()

In [6]:
classes [3], paragraphs [3]

(False,
 'for example our privacy policy talks about what information we collect and how this affects you it also explains the steps we take to protect your privacy like building our services so delivered messages arent stored by us and giving you control over who you communicate with on our services')

In [7]:
# Il faut mieux diviser le dataset une seule fois et stocker les deux datasets dans deux fichiers séparés
# Puisque si vous appliquez ça à chaque fois les échantillons seront différents et donc
# vous ne pouvez pas comparer entre les différents modèles si vous appliquez des entraînement séparés
# X1_train, X1_test, y_train, y_test = train_test_split(paragraphs, classes, test_size=0.2, random_state=42)
X1_train, X1_test, y_train, y_test = train_test_split(df["paragraph"],df["classe"],test_size=0.2, random_state=42)


count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(X1_train)
X_test = count_vectorizer.transform(X1_test)

In [8]:
#entrainement Naive Bayes
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

MultinomialNB()

In [9]:
#test Naive Bayes
Y_naive_bayes = naive_bayes.predict(X_test)

In [17]:
print(classification_report(y_test, Y_naive_bayes))

              precision    recall  f1-score   support

       False       0.70      0.64      0.67        33
        True       0.82      0.86      0.84        63

    accuracy                           0.78        96
   macro avg       0.76      0.75      0.75        96
weighted avg       0.78      0.78      0.78        96



In [18]:
# j'ai modifié votre fonction
# Pour naive bayes, la prédiction doit passer par le même pipeline
# Dans votre version précédente, vous avez utilisé un autre pipeline que celui 
# utilisé dans l'entraînement

def predict(text_str):
    x_input = count_vectorizer.transform([text_str])
    return naive_bayes.predict(x_input)[0]

In [19]:
predict("we may share personal data with card networks and payment processors")

True

In [20]:
predict("if the new version reduces your rights or increases your responsibilities well post it on the policy updates or privacy statement page of our website at least 21 days before it becomes effective")

False

In [21]:
predict("prezi inc will be the controller of your personal information provided to or collected by or for or processed in connection with our services the terms of use together with the other policies and agreements listed therein are between you and prezi inc in the case of content information you provide see content information section below you control such information you should be solely liable for it and it is outside our control")

True

In [28]:
# convert categoricals
from sklearn.preprocessing import LabelEncoder

encoders = {}
for column in ['true', 'fase']:
    categorical_convert = LabelEncoder()
#     X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

{'true': LabelEncoder(), 'fase': LabelEncoder()}

In [29]:
import joblib
joblib.dump(X_train, "./X_train.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(naive_bayes, "./naive_bayes.joblib", compress=True)

['./naive_bayes.joblib']